In [ ]:
# Importuri esențiale
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import joblib

# Încarcă datele (asigură-te că fișierul este în același folder cu notebook-ul)
df = pd.read_csv("nasa.csv")  # redenumește fișierul descărcat dacă e cazul
df.head()

# Informații generale
df.info()
df.describe()
df.isnull().sum()  # verifică valori lipsă

# Conversie coloane booleene
df['Hazardous'] = df['Hazardous'].astype(int)

# Elimină coloanele irelevante dacă există
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# Separare X și y
X = df.drop("Hazardous", axis=1)
y = df["Hazardous"]

# Normalizare
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Distribuția targetului
sns.countplot(x=y)
plt.title("Distribuția asteroizilor periculoși vs. non-periculoși")
plt.show()

# Matrice de corelație
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Corelația dintre caracteristici")
plt.show()

# Împărțirea setului în train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predicții și evaluare
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Matrice de confuzie:\n", confusion_matrix(y_test, y_pred))

scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
print("Cross-validation accuracy: ", scores.mean())

joblib.dump(model, "asteroid_model.pkl")
joblib.dump(scaler, "scaler.pkl")
